In [1]:
# import libraries
import pandas as pd
from pytrends.request import TrendReq

In [2]:
pytrends = TrendReq(hl='it-IT', tz=15)

In [3]:
key_words = [
    "Vacanze",
    "Viaggi",
    #"Gita",
    #"Google Maps",
    #"Traffico",
    #"Traffico Milano",
    #"Mare",
    #"Montagna",
    #"Lago Garda",
    #"Lago Como",
    #"Lago Iseo",
    #"Lago Lugano",
    #"Lago Maggiore",
    #"Lago Orta",
    #"Gardaland",
    #"Verona",
    #"Venezia",
    #"Svizzera",
    #"Austria",
    #"Trentino-South Tyrol",
    #"Liguria",
    #"Riviera Romagnola",
    #"Agrate Brianza",
    #"Provincia Monza Brianza",
    #"Santo Ambrogio",
    #"Halloween",
    #"Natale",
    #"Pasqua",
    #"Ferragosto",
    #"Capodanno",
    #"Fiera",
    #"Sagra",
    #"Manifestazione",
    #"Festa",
    #"Evento",
    #"Carnevale venezia",
    #"Eventi Monza",
    #"Feste Monza",
    #"Partita milan",
    #"Partita inter",
    #"Partita juventus",
    #"champions league",
    #"Partita calcio",
    #"Formula 1",
    #"Previsioni del tempo",
    #"Pioggia",
    #"Neve",
    #"Meteo",
    #"Meteo Agrate",
    #"autostrada A4",
    #"Acqua e farina agrate",
    #"Gruppo ethos ristoranti",
    #"Ristoranti Agrate",
    #"Pizza",
    #"Sushi",
    #"Ristorante"
]

In [4]:
# get dayly data from google trends
for key_word in key_words:
    pytrends.build_payload([key_word], cat=0, timeframe='today 9-m', geo='IT-25', gprop='')
    df = pytrends.interest_over_time()
    df.drop(columns=['isPartial'], inplace=True)
    df.to_csv('data/' + key_word + '.csv')

ResponseError: The request failed: Google returned a response with code 400.

In [6]:
df = pd.read_csv('data/Vacanze.csv', index_col=0)
df

,Vacanze
date,
2022-09-12,76
2022-09-13,61
2022-09-14,58
2022-09-15,53
2022-09-16,66
...,...
2022-12-04,82
2022-12-05,67
2022-12-06,56
